<a href="https://colab.research.google.com/github/mitkrieg/dl-assignment-1/blob/main/assignment1_practical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
import gzip
import typing as T
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load Data using Pytorch

In [2]:

PATH = '/content/drive/MyDrive/Fall 2024/Deep Learning/Assignment 1/data'



In [3]:
class FasionMINSTDataset(Dataset):
  def __init__(self, path: str, kind: str, transform=None, target_transform=None) -> None:
    self.transform = transform
    self.target_transform = target_transform
    self.labels, self.images = self._load_data(path, kind)
    self.labels_map = {
            0: "T-Shirt",
            1: "Trouser",
            2: "Pullover",
            3: "Dress",
            4: "Coat",
            5: "Sandal",
            6: "Shirt",
            7: "Sneaker",
            8: "Bag",
            9: "Ankle Boot",
        }

  def _load_data(self, path: str, kind: str) -> T.Tuple[np.ndarray, np.ndarray]:
    with gzip.open(path + f'/{kind}-labels-idx1-ubyte.gz', 'rb') as lable_file:
      lbls = np.frombuffer(lable_file.read(), dtype=np.int8, offset=8)
      lbls = np.copy(lbls)
    with gzip.open(path + f'/{kind}-images-idx3-ubyte.gz', 'rb') as lable_file:
      imgs = np.frombuffer(lable_file.read(), dtype=np.uint8, offset=16).reshape(len(lbls), 1, 28, 28)
      imgs = (np.copy(imgs) / 255).astype(np.float32)
    return lbls, imgs

  def __len__(self) -> int:
    return self.labels.size

  def __getitem__(self, index) -> T.Union[T.Dict[str, T.Any],T.List[T.Dict[str, T.Any]]]:
    label = torch.tensor(self.labels[index], dtype=torch.long)
    img = torch.tensor(self.images[index])

    if self.target_transform:
      label = self.target_transform(label)
    if self.transform:
      img = self.transform(img)


    return img, label

  def show_img(self, index):
    label = self.labels[index]
    img = self.images[index]
    plt.imshow(img.reshape(28,28), cmap='gray')
    plt.title(self.labels_map[label])


train = FasionMINSTDataset(PATH, 'train')
test = FasionMINSTDataset(PATH, 'test')

In [4]:
batch = 32
trainloader = DataLoader(train, batch, shuffle=True, num_workers=2)
testloader = DataLoader(test, batch, shuffle=True, num_workers=2)

In [5]:
class Lenet5(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1)
    self.avg_pool1 = nn.AvgPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1)
    self.avg_pool2 = nn.AvgPool2d(2, 2)
    self.fc1 = nn.Linear(16*4*4, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84,10)

  def forward(self, x):
    x = F.tanh(self.conv1(x))
    x = self.avg_pool1(x)
    x = F.tanh(self.conv2(x))
    x = self.avg_pool2(x)
    x = torch.flatten(x, 1)
    x = F.tanh(self.fc1(x))
    x = F.tanh(self.fc2(x))
    x = self.fc3(x)
    return x

net = Lenet5(batch)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=1e-3, momentum=0.9)

In [8]:
for epoch in range(5):
  total_loss = 0
  for i, data in enumerate(trainloader):
    inputs, labels = data

    optimizer.zero_grad()
    outputs = net(inputs)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
    if i % 64 == 63:
      print(f'Epoch #{epoch + 1}, Batch #{i + 1}: {total_loss / 64}')
      total_loss = 0


Epoch #1, Batch #64: 2.3044502213597298
Epoch #1, Batch #128: 2.2952739894390106
Epoch #1, Batch #192: 2.282403290271759
Epoch #1, Batch #256: 2.2680815905332565
Epoch #1, Batch #320: 2.244790866971016
Epoch #1, Batch #384: 2.2054286375641823
Epoch #1, Batch #448: 2.128204472362995
Epoch #1, Batch #512: 1.9951631464064121
Epoch #1, Batch #576: 1.7966137751936913
Epoch #1, Batch #640: 1.6062350161373615
Epoch #1, Batch #704: 1.4634484015405178
Epoch #1, Batch #768: 1.3225453738123178
Epoch #1, Batch #832: 1.2668433412909508
Epoch #1, Batch #896: 1.1951732262969017
Epoch #1, Batch #960: 1.1503965305164456
Epoch #1, Batch #1024: 1.1562119079753757
Epoch #1, Batch #1088: 1.0958245769143105
Epoch #1, Batch #1152: 1.0464043989777565
Epoch #1, Batch #1216: 1.0475705387070775
Epoch #1, Batch #1280: 1.0401470763608813
Epoch #1, Batch #1344: 1.0069887405261397
Epoch #1, Batch #1408: 0.9924458032473922
Epoch #1, Batch #1472: 0.9966997699812055
Epoch #1, Batch #1536: 0.92018848285079
Epoch #1, Bat

In [9]:
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the {len(test)} test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 79 %
